<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama/blob/main/RAG_RespondendoPergunta_Llama2_7B_4bit_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Respondendo Perguntas sobre Textos Longos usando Llama v2.0, Longchain, Chroma e Transformers by HuggingFace

**Toda a execução ocorre no Google Colaboratory.**

Pré-requisitos:
- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Referências**

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

**Lista dos modelos:**

https://huggingface.co/models


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Imprime linhas menores.

In [ ]:
def print_linhas_menores(texto, tamanho=120):
  for i in range(0, len(texto), tamanho):
    print(texto[i:i+tamanho])

# 1 - Instalação das bibliotecas

Biblioteca para manipular pdf

https://pypi.org/project/pypdf/

In [ ]:
!pip install pypdf==3.16.4

Bibliota de dependência para manipular os embeddings pelo Langchain.

https://pypi.org/project/sentence-transformers/

In [ ]:
!pip install sentence_transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 947.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

Biblioteca que persiste os embeddings e realiza busca semântica.

https://pypi.org/project/chromadb/

In [ ]:
!pip install chromadb==0.4.15

Biblioteca para realizar a divisão por token.

In [ ]:
!pip install tiktoken==0.5.1

Bibioteca LangChain é um framework de código aberto para o desenvolvimento de aplicações usando modelos de linguagem grandes.

https://pypi.org/project/langchain/

In [ ]:
!pip install langchain==0.0.323

Dependências do xformers

https://pypi.org/project/lmdb/

In [ ]:
!pip install lmdb==1.4.1
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 torchtext==0.15.2+cpu torchdata==0.6.1 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp310-cp310-linux_x86_64.whl (2267.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.22.post7 requires torch==2.1.0, but you have torch 2.0.1+cu118 which is incompatible.


Permite maior velocidade e menor consumo de memória nos transformers.

https://pypi.org/project/xformers/

In [ ]:
!pip install xformers==0.0.22.post7

  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchtext 0.15.2+cpu requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have to

O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

https://huggingface.co/blog/4bit-transformers-bitsandbytes

https://pypi.org/project/bitsandbytes/

In [ ]:
!pip install bitsandbytes==0.41.1

Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

https://pypi.org/project/accelerate/

In [ ]:
!pip install accelerate==0.24.0

A Biblioteca A Biblioteca Transformers fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração para Processamento de linguagem natural, Visão computacional, Áudio, etc.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
# Instala a última versão da biblioteca
# !pip install transformers

# A última versão do huggingface apresenta um problema:
# UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1`
# https://discuss.huggingface.co/t/help-with-llama-2-finetuning-setup/50035
# Usar a versão 4.31.0

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.31.0

A Biblioteca huggingface-cli fornece vários comandos para interagir com o Hugging Face Hub a partir da linha de comando. Um desses comandos é o login, que permite aos usuários se autenticarem no Hub usando suas credenciais.

In [ ]:
!pip install huggingface_hub==0.18.0

# 2 - Carregando o LLM



## 2.1 - Login no huggingface

- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.

Insira o token quando solicitado e depois digite Y para adicionar as credenciais.

In [ ]:
# !huggingface-cli login

Se o seu notebook não for público e não desejar incluir o token de acesso toda vez que for executar o notebook preencha o método save_token.

In [ ]:
from huggingface_hub.hf_api import HfFolder

ACCESS_TOKEN  = 'COLOQUE O SEU TOKEN DE ACESSO AQUI'

HfFolder.save_token(ACCESS_TOKEN)

Mostrando o usuário conectado

In [ ]:
# !huggingface-cli whoami

## 2.2 - Nome do LLM

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/meta-llama/Llama-2-7b-hf
  - https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-chat-hf

In [ ]:
# Roda com 4 bits e 8 bits
#nome_modelo_llm = "meta-llama/Llama-2-7b-hf"
nome_modelo = "meta-llama/Llama-2-7b-chat-hf"

# Roda com 4 bits
#nome_modelo = "meta-llama/Llama-2-13b-hf"
# nome_modelo = "meta-llama/Llama-2-13b-chat-hf"

# Não roda pois exige GPU A100 e mais espaço em disco
#nome_modelo = "meta-llama/Llama-2-70b-hf"
# nome_modelo = "meta-llama/Llama-2-70b-chat-hf"

## 2.3 - Carrega o tokenizador do LLM

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

Carregando o tokenizador meta-llama/Llama-2-7b-chat-hf da comunidade...


## 2.4 - Carregando o Modelo LLM

Carregando o **modelo** da comunidade Huggingface.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

Carregamento LLama 2 com 4 bits

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import time
import torch

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# BitsAndBytes é um framework com funções customizadas para
# otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
quantization_config = BitsAndBytesConfig(
   load_in_4bit=True, # Habilita a quantização de 4 bits para comprimir o modelo
   bnb_4bit_quant_type="nf4", # Define o tipo de dados de quantização nas camadas (`fp4` e `nf4`).
   bnb_4bit_use_double_quant=True, # Quantização aninhada, onde as constantes de quantização da primeira quantização são quantizadas novamente.
   bnb_4bit_compute_dtype=torch.bfloat16 # # Os gradientes dos pesos são computados em 16-bit. Define o tipo computacional que pode ser diferente do tempo de entrada. Por exemplo, as entradas podem ser fp32, mas a computação pode ser definida como bf16 para acelerações.
)

# Carrega o modelo
model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             trust_remote_code=True, # Carrega de um repositório confiável
                                             quantization_config=quantization_config,
                                             device_map="auto"
                                             )

# Coloca o modelo e modo avaliação
model.eval()

print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregando o modelo meta-llama/Llama-2-7b-chat-hf da comunidade...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Tempo de carregamento do modelo LLM:  0:01:15 (h:mm:ss)


Carregamento LLama 2 com 8 bits

In [ ]:
# # Importando as bibliotecas do Modelo
# from transformers import AutoModelForCausalLM
# import time

# # Guarda o tempo de início do carregamento do modelo
# tempo_inicio = time.time()

# # Carregando o Modelo da comunidade
# print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# model = AutoModelForCausalLM.from_pretrained(nome_modelo,
#                                              #torch_dtype=torch.float16, #default
#                                              trust_remote_code=True,   # Carrega de um repositório confiável
#                                              load_in_8bit=True,
#                                              device_map="auto"
#                                              )

# # Coloca o modelo e modo avaliação
# model.eval()

# print("Tempo de carregamento do modelo:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

In [ ]:
print(model.config.max_position_embeddings)

4096


In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}



## 2.5 - Cria o pipeline usando Langchain

Cria o pipeline com a classe [HuggingFacePipeline](https://api.python.langchain.com/en/latest/llms/langchain.llms.huggingface_pipeline.HuggingFacePipeline.html) do langchain.

Passagem direta do pipeline Huggingface.

Configura o pipeline do Huggingface usando o modelo e tokenizador previamente carregado e passa para o HuggingFacePipeline do langchain.

In [ ]:
# Import das bibliotecas
from langchain.llms import HuggingFacePipeline
from transformers import GenerationConfig, pipeline

# Instância as configurações do modelo
generation_config = GenerationConfig.from_pretrained(nome_modelo)
print("Antes:",generation_config)
generation_config.max_new_tokens = 4096 #Preenche até um comprimento máximo especificado com o argumento max_lengthou até o comprimento de entrada máximo aceitável para o modelo se esse argumento não for fornecido.
generation_config.temperature = 0.2 # A temperatura é um parâmetro que controla a aleatoriedade da saída do LLM. Uma temperatura mais alta resultará em um texto mais criativo e imaginativo, enquanto uma temperatura mais baixa resultará em um texto mais preciso e factual.
generation_config.top_p = 0.75 # Top-p é mais dinâmico que top-k e é frequentemente usado para excluir resultados com probabilidades mais baixas. Portanto, se você definir p como 0,75, excluirá os 25% inferiores dos resultados prováveis.
generation_config.top_k = 5  # Top-k diz ao modelo para escolher o próximo token entre os 'k' tokens principais de sua lista, classificados por probabilidade.
#generation_config.do_sample = True # (Default True) Se definido como True, este parâmetro permite estratégias de decodificação como amostragem multinomial, amostragem multinomial de busca de feixe, amostragem Top-K e amostragem Top-p. Todas essas estratégias selecionam o próximo token da distribuição de probabilidade em todo o vocabulário com vários ajustes específicos da estratégia.
generation_config.repetition_penalty = 1.20 # Penaliza a repetição e visa evitar frases que se repetem sem nada de realmente interessante.
print("Depois:",generation_config)

# Configura o pipeline do HuggingFace
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # Langchain espera o texto completo
    generation_config=generation_config,
)

# Carrega o pipeline do Langchain
# https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
model_llm = HuggingFacePipeline(
    pipeline=pipe,
)

Antes: GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.31.0"
}

Depois: GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 4096,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.2,
  "top_k": 5,
  "top_p": 0.75,
  "transformers_version": "4.31.0"
}



In [ ]:
print(model_llm)

HuggingFacePipeline
Params: {'model_id': 'gpt2', 'model_kwargs': None, 'pipeline_kwargs': None}


## 2.6 - Nome do LM BERT

In [ ]:
# Modelos em Inglês
#nome_modelo_bert = "bert-large-cased"
#nome_modelo_bert = "bert-base-cased"

# Modelos em Português
nome_modelo_bert = "neuralmind/bert-large-portuguese-cased"
#nome_modelo_bert = "neuralmind/bert-base-portuguese-cased"

## 2.7 - Carregando o LM BERT

A classe HuggingFaceBgeEmbeddings realiza o download do BERT via HuggingFace.

Os modelos [BGE](https://python.langchain.com/docs/integrations/text_embedding/bge_huggingface) no HuggingFace são os melhores modelos de embeddings de código aberto. O modelo BGE é criado pela Academia de Inteligência Artificial de Pequim(*Beijing Academy of Artificial Intelligence*-BAAI) . BAAI é uma organização privada sem fins lucrativos envolvida em pesquisa e desenvolvimento de IA.

In [ ]:
# Import das bibliotecas
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carrega os embeddings diretamente do HuggingFace
model_lm = HuggingFaceBgeEmbeddings(
    model_name=nome_modelo_bert,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

print("Tempo de carregamento do modelo LM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Tempo de carregamento do modelo LM:  0:00:12 (h:mm:ss)


In [ ]:
print(model_lm)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
) model_name='neuralmind/bert-large-portuguese-cased' cache_folder=None model_kwargs={'device': 'cuda'} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: '


Carrega o tokenizador do bert

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo_bert + ' da comunidade...')

tokenizer_bert = AutoTokenizer.from_pretrained(nome_modelo_bert)

Carregando o tokenizador neuralmind/bert-large-portuguese-cased da comunidade...


# 3 - Pergunta em texto longo

Artigos que auxiliaram a criar o recuperador de texto de perguntas em texto longo.

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

## 3.1 - Carrega os Documentos

Carrega de PDF ou HTML.

Escolha uma das formas de carregar os dados.

Existem outros formas de carregamento.

https://python.langchain.com/docs/modules/data_connection/document_loaders/

### 3.1.1 - Carrega os documentos de PDF

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

####  Download do PDF


Download do PDF do livro "As Vítimas Algozes".

PDF: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977


Biblioteca: https://www.literaturabrasileira.ufsc.br/documentos/?id=142070


In [ ]:
# Import das bibliotecas
# import subprocess

# # As Vítimas Algozes
# urlpdf = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977'

# destino = 'arquivo1.pdf'

# # Executa o comando wget no prompt
# subprocess.call(["wget", urlpdf, "-O", destino])

#### Carrega o PDF



In [ ]:
# # Import das bibliotecas
# from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# # Define o diretório
# diretorio = '/content'

# # Cria o carregar dos documentos do diretório
# # Pode ser usado o PyPDFLoader para um arquivo
# carregador = PyPDFDirectoryLoader(diretorio)

# # Carrega os documentos
# documentos = carregador.load()

### 3.1.2 - Carrega os documentos da WEB

https://python.langchain.com/docs/integrations/document_loaders/web_base

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

#### Carrega o HTML

Carrega o HTML do livro "As Vítimas Algozes".

HTML: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979

Biblioteca: https://www.literaturabrasileira.ufsc.br/documentos/?id=142070



In [ ]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

urlhtml = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# Cria o carregador da página
carregador = WebBaseLoader(urlhtml)

# Carrega os documentos
documentos = carregador.load()

Exibe parte dos dados carregados do PDF ou HTML.

In [ ]:
print("Quantidade de documentos(páginas):", len(documentos))
pagina = 0
print("Trecho página(", pagina, ") :", documentos[pagina].page_content[0:500])
print()
print("Medadados:", documentos[0].metadata)

Quantidade de documentos(páginas): 1
Trecho página( 0 ) : 















As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma ven

Medadados: {'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'title': 'As vítimas algozes - Joaquim Manuel de Macedo', 'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.'}


## 3.2 - Divide e sobrepõe os documentos em chuks

Carrega o documentos e realiza o divisão do documento em pedaços(chunks) e faz a sobreposição(overlap) para garantir o contexto semântico entre os pedaços.

Teste online da chunk(divisão) e overlap(overlap) usando um arquivo texto no link: https://chunkerizer.streamlit.app/

In [ ]:
# Import das bibliotecas
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import time

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

# Configura o divisor
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer_bert,
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    add_start_index = True, # Adiciona o índice de início do chunk no documento
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos documentos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(documentos)} documentos html em {tempo_final - tempo_inicio} segundos!")
print(f"Quantidade de chunks: {len(chunks)}")

Carregando e dividindo 1 documentos html em 2.702892780303955 segundos!
Quantidade de chunks: 481


Mostra alguns chunks. A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [ ]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)
    # Documento HTML
    print('chunk #',i,' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    # Documento PDF
    # print('chunk #',i,' Doc:',chunk.metadata['source'],' Página:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 217  qtde token : 25  start_index: 16

As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo
-----------------------------------------------------------------------
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1429  qtde token : 220  start_index: 235

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma venda perto da fazenda: é a
parasita que se apega à árvore; pior que isso, é a inimiga hipócrita que rende
vassalagem à sua vítima.
A venda de que falo

In [ ]:
maior_chunk_token = 0
maior_chunk_token_bert = 0
maior_chunk_character = 0

for i, chunk in enumerate(chunks):
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)

    # Documento HTML
    print('chunk #',i,' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    # Documento PDF
    #print('chunk #',i,' Doc:',chunk.metadata['source'],' Página:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert),' start_index:', chunk.metadata.get('start_index') )

    # Procura os maiores valores
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(tokens_bert) > maior_chunk_token_bert:
      maior_chunk_token_bert = len(tokens_bert)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk token bert:", maior_chunk_token_bert)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 217  qtde token : 25  qtde token bert: 50  start_index: 16
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1429  qtde token : 220  qtde token bert: 361  start_index: 235
chunk # 2  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1530  qtde token : 253  qtde token bert: 394  start_index: 1357
chunk # 3  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1602  qtde token : 250  qtde token bert: 403  start_index: 2605
chunk # 4  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1623  qtde token : 244  qtde token bert: 406  start_index: 3902
chunk # 5  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1552  qtde token : 237  qtde token ber

## 3.3 - Armazena os chunks

Utiliza o Chroma um banco de dados de embeddings de código aberto e leve.

https://www.trychroma.com/

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

In [ ]:
# Import das bibliotecas
from langchain.vectorstores import Chroma

# Armazena os chunks usando o Chroma
vectorstore = Chroma.from_documents(documents=chunks, embedding=model_lm, persist_directory="./bancodedados", )

## 3.4 - Função de envio de perguntas ao LLM

In [ ]:
# Import das bibliotecas
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
import torch
import gc

def avaliarContexto(texto):

  # Cria o texto de prompt
  prompt_template = """Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido.
Contexto: {context}
>>PERGUNTA<< {question}
>>RESPOSTA<<"""

  # Cria o prompt
  # As variáveis "context" e "question" não podem ser alteradas
  # O parâmetro "context" recebe o contexto do carregado e armazenado no Chroma
  # O parâmetro "question" recebe a pergunta realizada sobre o contexto.
  prompt = PromptTemplate(input_variables=["context", "question"],
                          template=prompt_template)

  # Instancia o chain
  chain_type_kwargs = {"prompt": prompt}
  chain = RetrievalQA.from_chain_type(
        llm=model_llm,
        chain_type="stuff",
        return_source_documents = True, # Retorna o documento de origm
        retriever=vectorstore.as_retriever(), # Passa o contexto(livro carregado)
        #retriever=vectorstore.as_retriever(search_kwargs={'k':1}),
        chain_type_kwargs=chain_type_kwargs
    )

  # Executa o prompt no llm
  resultado = chain({"query":texto})

  # Esvazia a memória
  del chain
  gc.collect()
  torch.cuda.empty_cache()

  return resultado

## 3.5 - Algumas perguntas usando o contexto

Guarda o tempo de início de realização das perguntas

In [ ]:
# Importando as bibliotecas do Modelo
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

### Perguntas sobre o tempo

In [ ]:
texto = "Qual o período que ocorreu a história do texto?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 O período que ocorreu a história do texto é o período colonial.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Considerando o contexto em qual período que ocorreram os fatos?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os fatos narrados ocorreram na época imperial brasileira, no final do século XIX.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com de

### Perguntas sobre personagens

In [ ]:
texto = "Qual o nome de todos os personagens do texto?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Simeão, Eufêmia, O Barbudo, Paulo Borges, Esméria, o curandeiro e a defunta.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Considerando o contexto qual o nome de todos os personagens da história?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Simeão, Eufêmia, O Barbudo, Paulo Borges, Esméria, Marido Adúlter

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Quem são os principais personagens do contexto?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os principais personagens do contexto são:

1. Deus
2. Senhores (ou proprietários)
3. Escravos
4. Esposas/filhas dos se
nhores
5. Oppressores (ou responsáveis pela escravidão)

Origem: page_content='face de que Deus vos direis senhores de homens, que são homens como vós, e de\nque vos intitulais donos, senhores, árbitros abso\xadlutos?... E depois com esses\nescravos ao pé de vós, em torno de vós, com esses miseráveis degradados pela\ncondição violentada, engolfados nos ví\xadcios mais torpes, materializados,\ncorruptos, apodrecidos na escravidão, pestíferos pelo viver no pantanal da\npeste e tão vis, tão perigosos postos em contacto convosco, com vossas esposas,\ncom vossas filhas, que podereis esperar desses escravos, do seu contacto\nobrigado, da sua influência fatal?... Oh! Bani a escravidão!... A escravidão é\num crime da sociedade escravagista, e a escravidão se vinga desmoralizando,\nenvenenando”, desonrando, empestando, assassinando seus opressores. Oh!... Bani\na escra\xadvid

In [ ]:
texto = "Quais os nomes dos principais personagens do contexto?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os principais personagens do contexto são:

1. God (Deus)
2. Masters (senhores)
3. Slaves (escravos)
4. Wives (esposas)

5. Daughters (filhas)

Origem: page_content='As vítimas algozes - Joaquim Manuel de Macedo\n\n\n\nFonte: Biblioteca Digital de Literatura de Países Lusófonos\n\nLITERATURA BRASILEIRA \nTextos literários em\nmeio eletrônico\nAs\nVítimas-Algozes, de Joaquim Manuel de Macedo' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 16, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Considerando o contexto quem são os principais personagens da história?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os principais personagens da história são Simeão, Eufêmia, o Barbudo, Esméria e Paulo Borges.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}

In [ ]:
texto = "Considerando o contexto qual o nome dos principais personagens da história?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Simeão, Paulo Borges, Eufêmia, O Barbudo, Esméria.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome do pai da Angêlica?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 O nome do pai da Angêlica é Simeão.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome do pai da Florinda?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 O nome do pai da Florinda é Simeão.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também a que

In [ ]:
texto = "Qual o nome do pai do Domingo?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 O nome do pai do Domingo é Simeão.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome da mãe da Angêlica?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Eufêmia

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome da mãe da Florinda?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 A mãe da Florinda era a Angélica.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também a que l

In [ ]:
texto = "Qual o nome da mãe do Domingo?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Eufêmia

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome da mãe do Simeão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Eufêmia

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome da pessoa casada com a Angêlica?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Simeão

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome da pessoa casada com a Florinda?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Lucinda

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também a que lhe era dirigida e que dizi

In [ ]:
texto = "Qual o nome da pessoa casada com o Domingo?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Eufêmia

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome da pessoa casada com o Simeão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Esméria

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome dos filhos da Angêlica?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Os nomes dos filhos da Angêlica são Simeão, Simão e Judá.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome dos filhos da Florinda?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Lucinda é a mãe dos filhos da Florinda.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também a

In [ ]:
texto = "Qual o nome dos filhos do Domingo?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Os nomes dos filhos do Domingo são Simeão, José e Judá.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome dos filhos do Simeão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 O nome dos filhos do Simeão é Paulo e Ezequiel.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "A Angêlica é filha de quem?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 A Angêlica é filha de Simeão.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "A Florinda é filha de quem?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 A Florinda é filha de Cándida e Liberato.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também

In [ ]:
texto = "O Domingo é filho de quem?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 É filho de Simeão.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também a que lhe era dirigida

In [ ]:
Dtexto = "O Simeão é filho de quem?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 É filho de Simeão.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também a que lhe era dirigida

In [ ]:
texto = "Quais os nomes dos casais da história?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Simeão, Esméria, Paulo Borges e Eufêmia são os nomes dos personagens principais da história.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


### Perguntas diversas

In [ ]:
texto = "Liste para mim as palavras que estão relacionadas com escravidão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Claro! Aqui estão algumas palavras relacionadas com a escravidão:

1. Escravo - Um ser humano que está em serviço de ou
tro pessoa ou instituição, geralmente contra sua vontade.
2. Esclavatura - O sistema social e econômico basado na explor
ação e explotação de trabalhadores escravos.
3. Cativeiro - O estado de ser uma pessoa escrava, ou seja, estar sob o con
trole e domínio de outra pessoa ou instituição.
4. Trabalho forçado - O trabalho realizado por uma pessoa contra sua von
tade, geralmente por meio da coerção ou ameaça.
5. Comércio de escravos - O comércio de seres humanos como bens ou merca
dorias, normalmente para fins de exploração laboratorial.
6. Abolicionismo - Uma teoria política e social que busca abol
ir a escravidão e promover a igualdade entre diferentes grupos étnicos e racialidades.
7. Emancipação - O processo de li
bertar uma pessoa ou grupo de escravidão, usualmente através de leis ou acordos políticos.
8. Escravocratía - Um termo u
tilizado para descrever sistemas

In [ ]:
texto = "Liste para mim a quantidade de  palavras que estão relacionadas com escravidão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Em "O Crime Horrível" de Machado de Assis, há 10 palavras relacionadas com a escravidão:

1. Escravo (3 vezes)
2. Cumpl
ice (2 vezes)
3. Força (2 vezes)
4. Ferido (2 vezes)
5. Perdido (2 vezes)
6. Sangue (2 vezes)
7. Saltar (1 vez)
8. Fazen
da (1 vez)
9. Carrasco (1 vez)
10. Correção (1 vez)

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'langua

In [ ]:
texto = "Considerando o contexto que palavras possuem relação com escravidão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As palavras "escravidão" e "slave" (escravo) são usadas em diferentes contextos históricos e culturais, mas em geral, e
las referem-se a uma situação social e econômica onde pessoas são tratadas como bens e são obrigadas a trabalhar contra 
sua vontade, muitas vezes sob ameaça de violência ou outras formas de coerção. Em Portugal, a palavra "escravidão" é usa
da principalmente para referir-se à época colonial when the country had colonies in Africa and Asia where enslavement wa
s practiced. In Brazil, the word "escravo" is used to refer to the period of slavery that existed there from the 16th ce
ntury until the late 19th century. The term can also be used more broadly to describe any situation where people are tre
ated unfairly or exploited, such as forced labor or human trafficking.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamen

In [ ]:
texto = "Considerando o contexto que palavras possuem relação semântica com escravidão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As palavras "escravidão" e "servo" estão relacionadas, pois ambas são termos que referem-se a uma situação social e eco
nômica em que uma pessoa é tratada como propriedade de outra, sem direitos e liberdades individuais. No contexto da hist
ória, Simeão é descrito como um escravo que foi ferido e abandonado pela fazenda, onde foi encontrado pelo curandeiro. P
ortanto, a relação entre as palavras "escravidão" e "servo" é relevante neste caso, pois ilustra a condição de Simeão co
mo um servo desprotegido e explorado.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou co

In [ ]:
texto = "Considerando o contexto que palavras estão relacionadas a escravidão mas deixa isto implicito?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Sim, é possível considerar que as palavras "senhores" e "donos" utilizadas na citação estão relacionadas à escravidão, 
mesmo sem explicitar o tema. Essas palavras foram usadas nas epístolas de São Paulo no Novo Testamento, onde ele utiliza
 o termo "dominos" (Gr. kyrio) para referir-se aos cristãos, que eram considerados como proprietários ou donos de outros
 credenciados por Jesus Cristo. No entanto, aqui está uma interpretación diferente, pois a frase "Bani a escravidão!" é 
usada para condenar a escravidão e denunciar a condição de exploração e opressão dos escravos.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\

Textos do trabalho do **Leandro da Silveira Dias**

In [ ]:
texto = "Considerando a palavra \"negro\" com alvo, identifique outras palavras no contexto que são semanticamente similares."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Com base nas informações proporcionadas, identificarei outras palavras semelhantes ao termo "negro". No entanto, é impo
rtante notar que o uso dessas palavras pode ser considerado ofensivo ou discriminatório, especialmente se aplicados a pe
ssoas de cor negra. É recomendável evitar o uso dessas palavras em situações formais ou professionais, optando por termo
s más neutros e inclusivos. Algumas das palavras similares ao termo "negro" incluem:

* Africano (ou Afro-American)
* Ne
grito/Negra (embora essas palavras também tenham sido usadas histicamente para referir-se a pessoas de cor negra de form
a discriminatória)
* Mulato/Mulata (ainda que essas palavras também tenham sido usadas para separar pessoas de origem mi
sta racial)
* Pardo (no Brasil) / Biracial (nos Estados Unidos)

É importante lembrar que a cor da pele é uma caracterís
tica biológica complexa e multifacetada, e que a terminologia utilizada para descrevê-la deve ser cuidadosa e respeitosa
.

Origem: page_content='si.\nSi

In [ ]:
texto = "Considerando a palavra alvo \"negro\", identifique outras palavras no contexto que são semanticamente similares."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As seguintes palavras são semanticamente similares à "negro" no contexto:

* Black (inglês)
* Noir (francês)
* Nero (it
aliano)
* Schwarz (alemão)
* Negro (português)

É importante notar que essas palavras podem ter conotações diferentes de
pendendo do contexto em que são usadas. Por exemplo, while "black" can have negative connotations in some contexts, it i
s also commonly used to refer to people of African descent without any pejorative intentions. Similarly, "noir" and "ner
o" are often used in a neutral or even positive sense to describe something that is dark or mysterious.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos 

In [ ]:
texto = " Na sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ liste as palavras possuem ligação com a palavra \'negro\'"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  As palavras "rancor" e "desejo" estão ligadas à palavra "negro", pois elas são relacionadas ao tema da escravidão e da
 opresão sofrida pelos negros.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 

In [ ]:
texto = " Na sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ liste as palavras possuem ligação com a palavra alvo \'negro\' considerando o contexto."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  Em relação à sentença mencionada, as palavras "branco" e "francês" estão ligadas ao alvo "negro" no contexto histórico
 e social da época, pois eram usadas para diferenciar e discriminar pessoas de origem africana. No passado, esses termos
 foram utilizados para caracterizar a raça humana de acordo com a coloração da pele, tornando a escravidão ainda mais od
iosa e injusta. Então, nas situações descritas na narrativa, a escrava é descrita como "branca", opretendente como "fran
cês" e a senhora como "negra". Essas designações raciais são problemáticas e ofensivas, pois perpetuam a ideologia da se
gregação e da discriminação baseada na cor das skinhas. É importante lembrar que a escravidão foi uma prática ilegal e i
mmoral que afetou milhões de pessoas ao longo da História, e que a luta contra a discriminação e a igualdade de direitos
 continua sendo fundamental para construir uma sociedade mais justa e inclusiva.

Origem: page_content='si.\nSimeão contara demais com as suas forças: fe

In [ ]:
texto = "Usando leitura distante realize a analise da sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ para encontrar e listar as palavras possuem ligação com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Com base nas informações obtidas através da leitura distante, podemos identificar as seguintes palavras que possuem lig
ação com a palavra alvo 'negro':

1. negócio (aparece duas vezes)
2. escravidão (aparece três vezes)
3. negro (aparece q
uatro vezes)
4. vergonha (aparece uma vez)
5. pundonor (aparece uma vez)
6. vingança (aparece uma vez)
7. desejo (aparec
e uma vez)
8. rancor (aparece uma vez)

Observe que essas palavras não são sempre usadas de forma explícita ou directa p
ara referir-se ao negro, mas simplesmente estão relacionadas à ideia de corpeiro ou escravidão.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do c

In [ ]:
texto = "Considerando o contexto use leitura distante para realizar a analise da sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ para encontrar e listar as palavras possuem ligação com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Claro! Aqui estão as palavras que possuem ligação com a palavra alvo 'negro' na sentença analisada:

1. negócio (aparec
e duas vezes)
2. vingança
3. desejo
4. rancor
5. escravidão
6. vergonha
7. pundonor
8. negro (aparece uma única vez)

En
tendi!

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https:/

In [ ]:
texto = "Usando leitura distante realize a analise da sentença: \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ e liste as palavras possuem relação semântica com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As palavras relacionadas com a palavra alvo 'negro' são:

* Em falta de pundonor e de vergonha (relacionadas com a idei
a de dignidade e decoro)
* Rancório (relacionada com a ideia de dor ou irritação)
* Desejo (relacionada com a ideia de i
ntenção ou voluntariado)
* Vingança (relacionada com a ideia de retaliação ou castigo)

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes -

Tempo final de execução das perguntas

In [ ]:
print("Tempo de execução das perguntas:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Tempo de execução das perguntas:  0:11:54 (h:mm:ss)
